# Sampling Over Waveform Uncertainty

In this tutorial, we demonstrate using this package to construct a prior of waveform uncertainty parameters and then using bilby to sample them. This notebook can be downloaded [here](https://github.com/RyanSR71/waveform_uncertainty/blob/main/docs/source/notebooks/WFU_Sampling_Tutorial.ipynb).

The following cell is everything we need to import to run this tutorial. We also import the `WaveformUncertainty` package, which will need to be installed first. See [Installation](https://waveformuncertainty.readthedocs.io/en/latest/installation.html#installation).

In [2]:
import os
import numpy as np
import bilby
import matplotlib.pyplot as plt
import lal
import pesummary
import scipy
from pesummary.gw.file.strain import StrainData
from pesummary.io import read
import requests
import WaveformUncertainty as wfu

## Setting Up the Prior

Here we set up a binary black hole (BBH) prior with bilby.

In [3]:
prior = bilby.core.prior.PriorDict()

prior['chirp_mass'] = bilby.gw.prior.Uniform(name='chirp_mass',latex_label=r'$\mathcal{M}_c$',minimum=25,maximum=100,unit=r'$\mathrm{M}_{\odot}$')
prior['mass_ratio'] = bilby.gw.prior.Uniform(name='mass_2',latex_label=r'$q$',minimum=0.125,maximum=1)
prior['chi_1'] = bilby.gw.prior.AlignedSpin(name='chi_1',latex_label=r'$\chi_{1}$',minimum=-1,maximum=1)
prior['chi_2'] = bilby.gw.prior.AlignedSpin(name='chi_2',latex_label=r'$\chi_{2}$',minimum=-1,maximum=1)
prior['luminosity_distance'] = bilby.gw.prior.UniformSourceFrame(name='luminosity_distance',latex_label=r'$\mathcal{L}_{D}$',minimum=100, maximum=1000, unit='Mpc')
prior['geocent_time'] = bilby.gw.prior.Uniform(name='geocent_time',latex_label=r'$t_{c}$',minimum=1126259462.3,maximum=1126259462.5,unit='s')
prior['psi'] = bilby.core.prior.DeltaFunction(name='psi',latex_label=r'$\psi$',peak=1.5707963267948966)
prior['phase'] = bilby.core.prior.Uniform(name='phase',latex_label=r'$\phi$',minimum=0,maximum=6.2831853071795865,boundary='periodic')
prior['theta_jn'] = bilby.core.prior.Sine(name='theta_jn',latex_label=r'$\theta_{JN}$')
prior['dec'] = bilby.core.prior.DeltaFunction(name='dec',latex_label=r'$\delta$',peak=-1.25781518)
prior['ra'] = bilby.core.prior.DeltaFunction(name='ra',latex_label=r'$\alpha_{r}$',peak=1.45617592)

Setting up the priors is a complex process as we need to define the Gaussian components, truncation boundaries, and frequency boundaries for the amplitude and phase correction parameters. Luckily, the functions in `WaveformUncertainty` handle the complicated and tedious computation for us. However, we still need to supply the functions plenty of information.

First, we need to choose a set of BBH parameters. These will be the same parameters (particularly the masses) as the BBH signal we aim to recover. This is because the boundaries on the prior in frequency space, $f_\mathrm{IM}$ and $f_\mathrm{light}$, depend on the mass of the signal. For real runs, this can be estimated. For our case, we know the mass exactly, so we can just plug it in.

In [4]:
injection_parameters = dict(
    chirp_mass = 36,
    mass_ratio = 0.75,
    chi_1 = 0.15,
    chi_2 = -0.05,
    luminosity_distance = 500,
    geocent_time = 1126259462.4,
    psi = 1.5707963267948966,
    phase = 3.1415926535897932,
    theta_jn = 1.5707963267948966,
    dec = -1.25781518,
    ra = 1.45617592
)

We now set up a `WaveformGeneratorWFU` object without frequency nodes or indexes to create a reference waveform. Using the frequency array of the WaveformGenerator, we interpolate power spectral density (PSD) data from [LIGO DCC](https://dcc.ligo.org/login/index.shtml) to match the waveform, which will be necessary for computing the match, $\frak{M}$.

In [5]:
reference_waveform = wfu.WaveformGeneratorWFU(
                    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
                    waveform_arguments=dict(waveform_approximant='IMRPhenomD', reference_frequency=50.0, catch_waveform_errors=True, 
                                            f_low = 20.0, f_high=1024.0),
                    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole, 
                    sampling_frequency=4096, 
                    duration=4, frequency_nodes=None,indexes=None
                )

# the following code interpolates the PSD data from 20 Hz (f_low) to 1024 Hz (f_high) onto the frequency array of the waveform generator
psd_data = np.loadtxt('https://dcc.ligo.org/public/0158/P1900011/001/GWTC1_GW170817_PSDs.dat',comments='#')
PSDs = np.interp(np.linspace(20,1024,len(reference_waveform.frequency_array)),psd_data[:,0][0:128129],psd_data[:,1][0:128129])

12:04 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


The last pieces we need to generate the priors are the standard deviations of the waveform differences between two waveform models, $\delta\mathcal{A}_\mu(f)$ and $\delta\phi_\mu(f)$. We can get these from a set of parameterized waveform differences. See [Parameterization and Waveform Differences](https://waveformuncertainty.readthedocs.io/en/latest/notebooks/Parameterization_Tutorial.html).

In [6]:
# downloading the file and saving to a folder
try:
    os.mkdir('tutorial_files')
except:
    pass
file = requests.get('https://github.com/RyanSR71/waveform_uncertainty/raw/refs/heads/main/files/BBH_parameterization_nsamples_1000.npy', allow_redirects=True)
open("tutorial_files/BBH_parameterization_nsamples_1000.npy", 'wb').write(file.content)

# loading the file
parameterization = np.load("tutorial_files/BBH_parameterization_nsamples_1000.npy",allow_pickle=True)

# calculating the means and standard deviations of the waveform differences
mean_amplitude_difference,amplitude_uncertainty,mean_phase_difference,phase_uncertainty,frequency_grid = wfu.uncertainties_from_parameterization(parameterization,linear=True,resolution=0.1)

We can now calculate the amplitude and phase correction priors. We choose $\gamma\%=97.5\%$ as our match boundary and 5 non-zero parameters (`nnodes`). We add these priors to the existing BBH prior.

In [7]:
prior,frequency_nodes,indexes = wfu.WFU_dA_prior(amplitude_uncertainty,frequency_grid,injection_parameters,
                                                 reference_waveform,PSDs,match_boundary=97.5,duration=4,
                                                 nnodes=5,prior=prior)

dA_1 Prior Complete
dA_2 Prior Complete
dA_3 Prior Complete
dA_4 Prior Complete
dA_5 Prior Complete
Amplitude Correction Prior Complete


In [8]:
prior,frequency_nodes,indexes = wfu.WFU_dphi_prior(phase_uncertainty,frequency_grid,injection_parameters,
                                                   reference_waveform,PSDs,match_boundary=97.5,duration=4,
                                                   nnodes=5,prior=prior)

dphi_1 Prior Complete
dphi_2 Prior Complete
dphi_3 Prior Complete
dphi_4 Prior Complete
dphi_5 Prior Complete
Phase Correction Prior Complete


If we look at the prior, we confirm that the new parameters have been added properly. All correction priors other than dphi_1-5 and dA_1-5, i.e. dA_-3 and dphi_7, are fixed at zero and will not be sampled during parameter estimation. Their purpose is to enforce the $f_\mathrm{IM}$ and $f_\mathrm{light}$ boundary conditions without modifying the waveform generator itself. In other words, we handle these boundary conditions through the prior rather than through the waveform model.

In [9]:
prior

{'chirp_mass': Uniform(minimum=25, maximum=100, name='chirp_mass', latex_label='$\\mathcal{M}_c$', unit='$\\mathrm{M}_{\\odot}$', boundary=None),
 'mass_ratio': Uniform(minimum=0.125, maximum=1, name='mass_2', latex_label='$q$', unit=None, boundary=None),
 'chi_1': bilby.gw.prior.AlignedSpin(a_prior=Uniform(minimum=0, maximum=1, name=None, latex_label=None, unit=None, boundary=None), z_prior=Uniform(minimum=-1, maximum=1, name=None, latex_label=None, unit=None, boundary=None), name='chi_1', latex_label='$\\chi_{1}$', unit=None, boundary=None, minimum=-1.0, maximum=1.0),
 'chi_2': bilby.gw.prior.AlignedSpin(a_prior=Uniform(minimum=0, maximum=1, name=None, latex_label=None, unit=None, boundary=None), z_prior=Uniform(minimum=-1, maximum=1, name=None, latex_label=None, unit=None, boundary=None), name='chi_2', latex_label='$\\chi_{2}$', unit=None, boundary=None, minimum=-1.0, maximum=1.0),
 'luminosity_distance': bilby.gw.prior.UniformSourceFrame(minimum=100.0, maximum=1000.0, cosmology='Pl

<br>

## Parameter Estimation Example

In this section, we will show how to use `WaveformUncertainty.WaveformGeneratorWFU` to perfom parameter estimation with and without the waveform uncertainty parameters.

For this example, we will set up two parameter estimation runs: one with the waveform uncertainty correction and one without. In [Posterior Analysis](https://waveformuncertainty.readthedocs.io/en/latest/notebooks/Posterior_Analysis_Tutorial.html), we can look at the differences between the result files and examine the effectiveness of the correction.

We start by defining an injection, which will remain fixed for both runs. We need to use the same injection parameters used to create the priors, so we will take a draw of the waveform uncertainty parameters and replace the BBH parameters with what we want.

In [10]:
PE_injection = prior.sample()
for key in injection_parameters.keys():
    PE_injection[key] = injection_parameters[key]
PE_injection

{'chirp_mass': 36,
 'mass_ratio': 0.75,
 'chi_1': 0.15,
 'chi_2': -0.05,
 'luminosity_distance': 500,
 'geocent_time': 1126259462.4,
 'psi': 1.5707963267948966,
 'phase': 3.141592653589793,
 'theta_jn': 1.5707963267948966,
 'dec': -1.25781518,
 'ra': 1.45617592,
 'dA_1': -0.017759778064530222,
 'dA_2': 0.0003511717381006798,
 'dA_3': 0.1932166624756403,
 'dA_4': 0.1408925985754294,
 'dA_5': 2.04631343028506,
 'dA_6': 0.0,
 'dA_7': 0.0,
 'dA_0': 0.0,
 'dA_-1': 0.0,
 'dA_-2': 0.0,
 'dA_-3': 0.0,
 'dphi_1': 0.002546134188737703,
 'dphi_2': 0.0842469092503529,
 'dphi_3': 1.2992935375827168,
 'dphi_4': 0.7234911842164733,
 'dphi_5': -1.085863983528491,
 'dphi_6': 0.0,
 'dphi_7': 0.0,
 'dphi_0': 0.0,
 'dphi_-1': 0.0,
 'dphi_-2': 0.0,
 'dphi_-3': 0.0}

Now we can set up the waveform generator that will supply the sampler with the injected waveform. We use [WaveformUncertainty.WaveformGeneratorWFU](https://waveformuncertainty.readthedocs.io/en/latest/WaveformGeneratorWFU.html) as our waveform generator function to incoporate the injected waveform uncertainty parameters into the waveform. We include the frequency_nodes and allow the sampler to use the amplitude and phase correction parameters, $\alpha$ and $\varphi$.

In [11]:
injected_waveform = wfu.WaveformGeneratorWFU(
                parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
                waveform_arguments=dict(waveform_approximant='IMRPhenomD', reference_frequency=50, catch_waveform_errors=True, 
                                        f_low = 20.0, f_high=1024.0),
                frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole, 
                sampling_frequency=4096, 
                duration=4,
                # waveform uncertainty arguments
                frequency_nodes=frequency_nodes,
                indexes=indexes
            )

12:08 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


With the waveform generator set up, we set the interferometer objects with the injected signal, which will be the same for both runs.

In [12]:
# setting up interferometers
ifos = bilby.gw.detector.InterferometerList(['H1', 'L1'])
ifos.set_strain_data_from_power_spectral_densities(
    sampling_frequency=4096, duration=4,
    start_time=PE_injection['geocent_time'] - 2)
ifo_injection = ifos.inject_signal(
    waveform_generator=injected_waveform,
    parameters=PE_injection)

12:08 bilby INFO    : Injected signal in H1:
12:08 bilby INFO    :   optimal SNR = 11.02
12:08 bilby INFO    :   matched filter SNR = 12.40+0.67j
12:08 bilby INFO    :   chirp_mass = 36
12:08 bilby INFO    :   mass_ratio = 0.75
12:08 bilby INFO    :   chi_1 = 0.15
12:08 bilby INFO    :   chi_2 = -0.05
12:08 bilby INFO    :   luminosity_distance = 500
12:08 bilby INFO    :   geocent_time = 1126259462.4
12:08 bilby INFO    :   psi = 1.5707963267948966
12:08 bilby INFO    :   phase = 3.141592653589793
12:08 bilby INFO    :   theta_jn = 1.5707963267948966
12:08 bilby INFO    :   dec = -1.25781518
12:08 bilby INFO    :   ra = 1.45617592
12:08 bilby INFO    :   dA_1 = -0.017759778064530222
12:08 bilby INFO    :   dA_2 = 0.0003511717381006798
12:08 bilby INFO    :   dA_3 = 0.1932166624756403
12:08 bilby INFO    :   dA_4 = 0.1408925985754294
12:08 bilby INFO    :   dA_5 = 2.04631343028506
12:08 bilby INFO    :   dA_6 = 0.0
12:08 bilby INFO    :   dA_7 = 0.0
12:08 bilby INFO    :   dA_0 = 0.0
1

Now we set up for the parameter estimation runs themselves. We start with the run that does not perform the waveform uncertainty correction.

We set up a new waveform generator object, which we will call "NHP_waveform". NHP represents "null-hypothesis", which means that this run assumes that there are no uncertainties. We often represent its likelihood with $\mathcal{L}_{\varnothing}(h|\theta)$, which does not include the $\alpha$ and $\varphi$ parameters. We use bilby's WaveformGenerator for this waveform.

In [13]:
NHP_waveform = bilby.gw.WaveformGenerator(
                parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
                waveform_arguments=dict(waveform_approximant='IMRPhenomD', reference_frequency=50, catch_waveform_errors=True, 
                                        f_low = 20.0, f_high=1024.0),
                frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole, 
                sampling_frequency=4096, 
                duration=4,
            )

12:08 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


Now we define the likelihood object and the sampler. Running the sampler will start the parameter estimation run, which will take hours to complete, so it will not be run here.

In [14]:
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifos,
    NHP_waveform,
    priors=prior,
    time_marginalization=False, 
    phase_marginalization=False, 
    distance_marginalization=False,
)

In [ ]:
result = bilby.run_sampler(
    likelihood, prior, sampler='nestle', 
    label="NHP_result",
    conversion_function=bilby.gw.conversion.generate_all_bns_parameters,
    nlive=5000, 
    dlogz=0.1,
    clean=True,
    maxiter=None,
)

With the null-hypothesis run complete, we now set up the waveform uncertainty correction run. We will call this new waveform generator "WFU_waveform", where WFU respresents "waveform uncertainty". The likelihood for this run is denoted with $\mathcal{L}(h|\theta,\alpha,\varphi)$. We need to use [WaveformUncertainty.WaveformGeneratorWFU](https://waveformuncertainty.readthedocs.io/en/latest/WaveformGeneratorWFU.html) as the waveform generator function to use the waveform uncertainty parameters.

In [15]:
WFU_waveform = wfu.WaveformGeneratorWFU(
                parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
                waveform_arguments=dict(waveform_approximant='IMRPhenomD', reference_frequency=50, catch_waveform_errors=True, 
                                        f_low = 20.0, f_high=1024.0),
                frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole, 
                sampling_frequency=4096, 
                duration=4,
                frequency_nodes=frequency_nodes,
                indexes=indexes
            )

12:08 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


The steps of setting up the likelihood and sampler are repeated from the null-hypothesis run:

In [16]:
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifos,
    WFU_waveform,
    priors=prior,
    time_marginalization=False, 
    phase_marginalization=False, 
    distance_marginalization=False,
)

In [ ]:
result = bilby.run_sampler(
    likelihood, prior, sampler='nestle', 
    label="WFU_result",
    conversion_function=bilby.gw.conversion.generate_all_bns_parameters,
    nlive=5000, 
    dlogz=0.1,
    clean=True,
    maxiter=None,
)

To see how analysis of the posteriors of these runs would be carried out, see [Posterior Analysis](https://waveformuncertainty.readthedocs.io/en/latest/notebooks/Posterior_Analysis_Tutorial.html).